# Homework 5: Conditional Probability Models


## 1. Introduction

In this homework we'll be investigating conditional probability models,
with a focus on various interpretations of logistic regression, with
and without regularization. Along the way we'll discuss the calibration
of probability predictions, both in the limit of infinite training
data and in a more bare-hands way. On the Bayesian side, we'll recreate
from scratch the Bayesian linear gaussian regression example we discussed
in lecture. We'll also have several optional problems that work through
many basic concepts in Bayesian statistics via one of the simplest
problems there is: estimating the probability of heads in a coin flip.
Later we'll extend this to the probability of estimating click-through
rates in mobile advertising. Along the way we'll encounter empirical
Bayes and hierarchical models. 

# 2. From Scores to Conditional Probabilities

Let's consider the classification setting, in which $\left(x_{1},y_{1}\right),\ldots,\left(x_{n},y_{n}\right)\in\mathcal{X}\times\left\{ -1,1\right\} $ are sampled i.i.d. from some unknown distribution. For a prediction
function $f:\mathcal{X}\to\Re$, we define the \textbf{margin }on an example
$\left(x,y\right)$ to be $m=yf(x)$. Since our class predictions
are given by $\textrm{sign}(f(x))$, we see that a prediction is correct iff
$m(x)>0$. It's tempting to interpret the magnitude of the score $\left|f(x)\right|$
as a measure of confidence. However, it's hard to interpret the magnitudes
beyond saying one prediction score is more or less confident than
another, and without any scale to this ``confidence score``, it's
hard to know what to do with it. In this problem, we investigate how
we can translate the score into a probability, which is much easier
to interpret. In other words, we are looking for a way to convert
score function $f(x)\in\Re$ into a conditional probability distribution
$x\mapsto p(y=1\mid x)$. 

In this problem we will consider $\textbf{margin-based losses}$, which
are loss functions of the form $\left(y,f(x)\right)\mapsto\ell\left(yf(x)\right)$,
where $m=yf(x)$ is called the $\textbf{margin}$. We are interested
in how we can go from an empirical risk minimizer for a margin-based
loss, $\hat{f}=\text{argmin}_{f\in\mathcal{F}}\sum_{i=1}^{n}\ell\left(y_{i}f(x_{i})\right)$,
to a conditional probability estimator $\hat{\pi}(x)\approx p(y=1\mid x)$.
Our approach will be to try to find a way to use the Bayes prediction function $f^{*}=\text{argmin}_{f}\mathbb{E}_{x,y}\left[\ell(yf(x)\right]$ to get the true
conditional probability $\pi(x)=p(y=1\mid x$), and then apply the
same mapping to the empirical risk minimizer. While there is plenty
that can go wrong with this ``plug-in`` approach (primarily, the
empirical risk minimizer from a limited hypothesis space $\mathcal{F}$
may be a poor estimate for the Bayes prediction function), it is at
least well-motivated, and it can work well in practice. And $\textbf{please
note}$ that we can do better than just hoping for success: if you have
enough validation data, you can directly assess how well ``calibrated``
the predicted probabilities are. This blog post has some discussion
of calibration plots: [https://jmetzen.github.io/2015-04-14/calibration.html](https://jmetzen.github.io/2015-04-14/calibration.html). 

It turns out it is straightforward to find the Bayes prediction function
$f^{*}$ for margin losses, at least in terms of the data-generating
distribution: For any given $x\in\mathcal{X}$, we'll find the best possible
prediction $\hat{y}$. This will be the $\hat{y}$ that minimizes
$$
\mathbb{E}_{y}\left[\ell\left(y\hat{y}\right)\mid x\right].
$$
If we can calculate this $\hat{y}$ for all $x\in\mathcal{X}$, then we will
have determined $f^{*}(x)$. We will simply take
$$
f^{*}(x)=\text{argmin}_{\hat{y}}\mathbb{E}_{y}\left[\ell\left(y\hat{y}\right)\mid x\right].
$$

Below we'll calculate $f^{*}$ for several loss functions. It will
be convenient to let $\pi(x)=p\left(y=1\mid x\right)$ in the work
below.

**2.1** Write $\mathbb{E}{y}\left[\ell\left(yf(x)\right)\mid x\right]$ in terms
of $\pi(x)$, $\ell(-f(x))$, and $\ell\left(f(x)\right)$. (Hint:
Use the fact that $y\in\left\{ -1,1\right\}$).

**Answer**:
\begin{eqnarray*}
\mathbb{E}{y}\left[\ell\left(yf(x)\right)\mid x\right]
& = & \ell\left(f(x)\right) p(y=1\mid x) + \ell\left(-f(x)\right) p(y=-1\mid x) \\
& = & \ell\left(f(x)\right) p(y=1\mid x) + \ell\left(-f(x)\right) (1-p(y=1\mid x)) \\
& = & \ell\left(f(x)\right) \pi(x) + \ell\left(-f(x)\right) (1-\pi(x))
\end{eqnarray*}

**2.2** Show that the Bayes prediction function $f^{*}(x)$ for the exponential
loss function $\ell\left(y,f(x)\right)=e^{-yf(x)}$ is given by 
$$
f^{*}(x)=\frac{1}{2}\ln\left(\frac{\pi(x)}{1-\pi(x)}\right),
$$
where we've assumed $\pi(x)\in\left(0,1\right)$. Also, show that
given the Bayes prediction function $f^{*}$, we can recover the conditional
probabilities by
$$
\pi(x)=\frac{1}{1+e^{-2f^{*}(x)}}.
$$
(Hint: Differentiate the expression in the previous problem with
respect to $f(x)$. To make things a little less confusing, and also
to write less, you may find it useful to change variables a bit: Fix
an $x\in\mathcal{X}$. Then write $p=\pi(x)$ and $\hat{y}=f(x)$. After substituting
these into the expression you had for the previous problem, you'll
want to find $\hat{y}$ that minimizes the expression. Use differential
calculus. Once you've done it for a single $x$, it's easy to write
the solution as a function of $x$.) 

**Answer**: Let's write $p=\pi(x)$ and $\hat{y}=f(x)$, then we have
$$
\frac{\partial \ell\left(y,f(x)\right)}{\partial \hat{y}}
= e^{-y \hat{y}} (-y)
$$
Then we can calculate the partial derivative with respect to expectation function,
\begin{eqnarray*}
\frac{\partial \mathbb{E}}{\partial \hat{y}}
& = & -e^{-\hat{y}} p + e^{\hat{y}} (1-p) \\
& = & 0
\end{eqnarray*}
By calculating $\hat{y}$, and replace $p$ with $\pi(x)$ we arrive at
$$
f^{*}(x)=\frac{1}{2}\ln\left(\frac{\pi(x)}{1-\pi(x)}\right),
$$
From the above equation and arrange the equation, we can find $\pi(x)$ as
\begin{eqnarray*}
\pi(x) & = & \frac{e^{\hat{y}}}{e^{-\hat{y}} + e^{\hat{y}}} \\
& = & \frac{1}{1+e^{-2f^{*}(x)}}
\end{eqnarray*}

2.3 Show that the Bayes prediction function $f^{*}(x)$ for the logistic
loss function $\ell\left(y,f(x)\right)=\ln\left(1+e^{-yf(x)}\right)$
is given by
$$
f^{*}(x)=\ln\left(\frac{\pi(x)}{1-\pi(x)}\right)
$$
and the conditional probabilities are given by
$$
\pi(x)=\frac{1}{1+e^{-f^{*}(x)}}.
$$
Again, we may assume that $\pi(x)\in(0,1)$.

**Answer**: Let's write $p=\pi(x)$ and $\hat{y}=f(x)$, then we have
$$
\frac{\partial \ell\left(y,f(x)\right)}{\partial \hat{y}}
= \frac{1}{1+e^{-y \hat{y}}} e^{-y \hat{y}} (-y)
$$
Then we can calculate the partial derivative with respect to expectation function,
\begin{eqnarray*}
\frac{\partial \mathbb{E}}{\partial \hat{y}}
& = & \frac{-e^{-\hat{y}}}{1+e^{-\hat{y}}} p + \frac{e^{\hat{y}}}{1+e^{\hat{y}}} (1-p) \\
& = & 0
\end{eqnarray*}
By calculating $\hat{y}$, and replace $p$ with $\pi(x)$ we arrive at
$$
\frac{1}{1+e^{\hat{y}}} p = \frac{1}{1+e^{-\hat{y}}} (1-p) \\
f^{*}(x)=\ln\left(\frac{\pi(x)}{1-\pi(x)}\right)
$$
From the above equation and arrange the equation, we can find $\pi(x)$ as
$$
\pi(x)=\frac{1}{1+e^{-f^{*}(x)}}.
$$

# 3. Logistic Regression

## 3.1 Equivalence of ERM and probabilistic approaches

In lecture we discussed two different ways to end up with logistic regression. 

$\textbf{ERM approach:}$ Consider the classification setting with input
space $\mathcal{X}=\Re^{d}$, outcome space $\mathcal{Y}_{\pm}=\left\{ -1,1\right\} $,
and action space $\mathcal{A}_{\Re}=\Re$, with the hypothesis space of linear score functions $\mathcal{F}_{\text{score}}=\left\{ x\mapsto x^{T}w\mid w\in\Re^{d}\right\} $.
Consider the margin-based loss function $\ell_{\text{logistic}}(m)=\log\left(1+e^{-m}\right)$
and the training data $\mathcal{D}=\left((x_{1},y_{1}),\ldots,(x_{n},y_{n})\right)$.
Then the empirical risk objective function for hypothesis space $\mathcal{F}_{\text{score}}$
and the logistic loss over $\mathcal{D}$ is given by

\begin{eqnarray*}
\hat{R}_{n}(w) & = & \frac{1}{n}\sum_{i=1}^{n}\ell_{\text{logistic}}(y_{i}w^{T}x_{i})\\
 & = & \frac{1}{n}\sum_{i=1}^{n}\log\left(1+\exp\left(-y_{i}w^{T}x_{i}\right)\right).
\end{eqnarray*}

$\textbf{Bernoulli regression with logistic transfer function:}$ Consider
the conditional probability modeling setting with input space $\mathcal{X}=\Re^{d}$,
outcome space $\mathcal{Y}_{0/1}=\left\{ 0,1\right\} $, and action space
$\mathcal{A}_{[0,1]}=[0,1]$, where an action corresponds to the predicted
probability that an outcome is $1$. Define the $\textbf{standard logistic
function}$ as $\phi(\eta)=1/\left(1+e^{-\eta}\right)$ and the hypothesis
space $\mathcal{F}_{\text{prob}}=\left\{ x\mapsto\phi(w^{T}x)\mid w\in\Re^{d}\right\} $.
Suppose for every $y_{i}$ in the dataset $\mathcal{D}$ above, we define
$y_{i}'=\begin{cases}
1 & y_{i}=1\\
0 & y_{i}=-1
\end{cases}$, and let $\mathcal{D}'$ be the resulting collection of $\left(x_{i},y_{i}'\right)$ pairs. Then the negative log-likelihood (NLL) objective function for $\mathcal{F}_{\text{prob}}$ and $\mathcal{D}'$ is give by 
\begin{eqnarray*}
\text{NLL}(w) & = & -\sum_{i=1}^{n}y_{i}'\log\phi(w^{T}x_{i})+\left(1-y_{i}'\right)\log\left(1-\phi(w^{T}x_{i})\right)\\
 & = & \sum_{i=1}^{n}\left[-y_{i}'\log\phi(w^{T}x_{i})\right]+\left(y_{i}'-1\right)\log\left(1-\phi(w^{T}x_{i})\right)
\end{eqnarray*}
If $\hat{w}_{\text{prob}}$ minimizes $\text{NLL}(w)$, then $x\mapsto\phi(x^{T}\hat{w}_{\text{prob}})$
is a maximum likelihood prediction function over the hypothesis space
$\mathcal{F}_{\text{prob}}$ for the dataset $\mathcal{D}'$. 

**3.1.1** $\textbf{Show that}$ $n\hat{R}_{n}(w)=\text{NLL}(w)$ for all $w\in\Re^{d}$.
And thus the two approaches are equivalent, in that they produce the
same prediction functions.

**Answer**: We are going to show two approaches are equivalent by comparing one training sample, and can natually extend to all training samples.

\begin{eqnarray*}
\text{NLL}(w) & = & -(y_{i}'\log\phi(w^{T}x_{i})+\left(1-y_{i}'\right)\log\left(1-\phi(w^{T}x_{i})\right))\\
& = & - \log (\phi(w^{T}x_{i})^{y_{i}'} \left(1-\phi(w^{T}x_{i})\right)^{1-y_{i}'} )
\end{eqnarray*}
- when $y_{i}'=1$, $\text{NLL}(w) = -\log (\phi(w^{T}x_{i}))$, 
- which is the same as $n\hat{R}_{n}(w)$; when $y_{i}'=0$, $\text{NLL}(w) = -\log (1-\phi(w^{T}x_{i}))$, which is the 

## 3.2 Numerical Overflow and the log-sum-exp trick

Suppose we want to calculate $\log\left(\exp(\eta)\right)$ for $\eta=1000.42$.
If we compute this literally in Python, we will get an overflow (try
it!), since numpy gets infinity for $e^{1000.42}$, and log of infinity
is still infinity. On the other hand, we can help out with some math:
obviously $\log\left(\exp(\eta)\right)=\eta$, and there's no issue. 

It turns out, $\log(\exp(\eta))$ and the problem with its calculation
is a special case of the [LogSumExp](https://en.wikipedia.org/wiki/LogSumExp)
function that shows up frequently in machine learning. We define
$$
\text{LogSumExp}(x_{1},\ldots,x_{n})=\log\left(e^{x_{1}}+\cdots+e^{x_{n}}\right).
$$

Note that this will overflow if any of the $x_{i}$'s are large (more
than 709). To compute this on a computer, we can use the $\textbf{log-sum-exp
trick}$. We let $x^{*}=\max\left(x_{1},\ldots,x_{n}\right)$ and
compute LogSumExp as
$$
\text{LogSumExp}(x_{1},\ldots,x_{n})=x^{*}+\log\left[e^{x_{1}-x^{*}}+\cdots+e^{x_{n}-x^{*}}\right].
$$

**3.2.1** Show that the new expression for LogSumExp is valid.

**Answer**: We show that 
\begin{eqnarray*}
\text{LogSumExp}(x_{1},\ldots,x_{n}) 
&=& \log\left(e^{x_{1}}+\cdots+e^{x_{n}}\right) \\
&=& \log\left(x^{*} \frac{e^{x_{1}}+\cdots+e^{x_{n}}}{x^{*}} \right) \\
&=& x^{*}+\log\left[e^{x_{1}-x^{*}}+\cdots+e^{x_{n}-x^{*}}\right]
\end{eqnarray*}

**3.2.2**: Show that $\exp\left(x_{i}-x^{*}\right)\in(0,1]$ for any $i$, and
thus the exp calculations will not overflow.

**Answer**: Since $x_{i}-x^{*} <= 0$, thus $\exp\left(x_{i}-x^{*}\right)\in(0,1]$, making the exp calculations never go overflow.

**3.2.3**: Above we've only spoken about the exp overflowing. However, the log
part can also have problems by becoming negative infinity for arguments
very close to $0$. Explain why the $\log$ term in our expression
$\log\left[e^{x_{1}-x^{*}}+\cdots+e^{x_{n}-x^{*}}\right]$ will never
be ``-inf``.

**Answer**: since $x^{*}$ is the maxmum values all all x_i's, meaning that $\left[e^{x_{1}-x^{*}}+\cdots+e^{x_{n}-x^{*}}\right]$ always larger than $1$. Then we have never have no issue ``-inf``. 

**3.2.4**: In the objective functions for logistic regression, there are expressions
of the form $\log\left(1+e^{-s}\right)$ for some $s$. Note that a naive implementation gives $0$ for $s>36$ and inf for $s<-709$.
Show how to use the numpy function [logaddexp](https://docs.scipy.org/doc/numpy/reference/generated/numpy.logaddexp.html)
to correctly compute $\log\left(1+e^{-s}\right)$. 

In [1]:
import numpy as np
s = 36
print(np.logaddexp(0, -s))
s = -709
print(np.logaddexp(0, -s))

2.319522830243569e-16
709.0


## 3.3 Regularized Logistic Regression

For a dataset $\mathcal{D}=\left((x_{1},y_{1}),\ldots,(x_{n},y_{n})\right)$
drawn from $\Re^{d}\times\left\{ -1,1\right\} $, the regularized
logistic regression objective function can be defined as
\begin{eqnarray*}
J_{\text{logistic}}(w) & = & \hat{R}_{n}(w)+\lambda\|w\|^{2}\\
 & = & \frac{1}{n}\sum_{i=1}^{n}\log\left(1+\exp\left(-y_{i}w^{T}x_{i}\right)\right)+\lambda\|w\|^{2}.
\end{eqnarray*}

**3.3.1**: Prove that the objective function $J_{\text{logistic}}(w)$ is convex.
You may use any facts mentioned in the [convex optimization notes](https://davidrosenberg.github.io/mlcourse/Notes/convex-optimization.pdf).

**Answer**:

**3.3.2**: Complete the $\text{f_objective}$ function in the skeleton code,
which computes the objective function for $J_{\text{logistic}}(w)$.
Make sure to use the log-sum-exp trick to get accurate calculations and to prevent overflow.

**Answer**:

**3.3.3**: Complete the $\text{fit_logistic_regression_function}$ in the skeleton
code using the $\texttt{minimize}$ function from $\texttt{scipy.optimize}$.
$\text{ridge_regression.py}$ from Homework 2 gives an example of
how to use the $\texttt{minimize}$ function. Use this function to train
a model on the provided data. Make sure to take the appropriate preprocessing
steps, such as standardizing the data and adding a column for the
bias term.

**Answer**:

**3.3.4**: Find the $\ell_{2}$ regularization parameter that minimizes the log-likelihood
on the validation set. Plot the log-likelihood for different values
of the regularization parameter. 

**Answer**:

**3.3.5**: Based on the Bernoulli regression development of logistic regression,
it seems reasonable to interpret the prediction $f(x)=\phi(w^{T}x)=1/\left(1+e^{-w^{T}x}\right)$
as the probability that $y=1$, for a randomly drawn pair $\left(x,y\right)$.
Since we only have a finite sample (and we are regularizing, which
will bias things a bit) there is a question of how well [calibrated](https://en.wikipedia.org/wiki/Calibration_(statistics) 
our predicted probabilities are. Roughly speaking, we say $f(x)$
is well calibrated if we look at all examples $\left(x,y\right)$
for which $f(x)\approx0.7$ and we find that close to $70\%$ of those
examples have $y=1$, as predicted... and then we repeat that for
all predicted probabilities in $\left(0,1\right)$. To see how well-calibrated
our predicted probabilities are, break the predictions on the validation
set into groups based on the predicted probability (you can play with
the size of the groups to get a result you think is informative).
For each group, examine the percentage of positive labels. You can
make a table or graph. Summarize the results. You may get some ideas
and references from [scikit-learn's discussion](http://scikit-learn.org/stable/modules/calibration.html). 

**Answer**:

# 4. Bayesian Logistic Regression with Gaussian Priors

Let's return to the setup described in Section 3.1 and, in particular, to the Bernoulli regression setting with logistic transfer function. We had the following hypothesis space of conditional
probability functions:
$$
\mathcal{F}_{\text{prob}}=\left\{ x\mapsto\phi(w^{T}x)\mid w\in\Re^{d}\right\} .
$$
Now let's consider the Bayesian setting, where we induce a prior on
$\mathcal{F}_{\text{prob}}$ by taking a prior $p(w)$ on the parameter $w\in\Re^{d}$. 

**4.1** For the dataset $\mathcal{D}'$ described in Section 3.1,give an expression for the posterior density $p(w\mid\mathcal{D}')$ in terms of the negative log-likelihood function 
\begin{eqnarray*}
\text{NLL}_{\mathcal{D}'}(w) & = & -\sum_{i=1}^{n}y_{i}'\log\phi(w^{T}x_{i})+\left(1-y_{i}'\right)\log\left(1-\phi(w^{T}x_{i})\right)
\end{eqnarray*}
and a prior density $p(w)$ (up to a proportionality constant is fine)

**Answer**:

**4.2** Suppose we take a prior on $w$ of the form $w\sim\mathcal{N}(0,\Sigma)$.
Find a covariance matrix $\Sigma$ such that MAP estimate for $w$
after observing data $\mathcal{D}'$ is the same as the minimizer of the regularized
logistic regression function defined in Section 3.3 (and prove it). (Hint: Consider minimizing the negative log posterior of $w$. Also, remember you can drop any terms from the objective
function that don't depend on $w$. Also, you may freely use results of previous problems.)

**Answer**:

**4.3** In the Bayesian approach, the prior should reflect your beliefs about
the parameters before seeing the data and, in particular, should be
independent on the eventual size of your dataset. Following this,
you choose a prior distribution $w\sim\mathcal{N}(0,I)$. For a dataset $\mathcal{D}$
of size $n$, how should you choose $\lambda$ in our regularized
logistic regression objective function so that the minimizer is equal
to the mode of the posterior distribution of $w$ (i.e. is equal to
the MAP estimator). 

**Answer**:

# 5 Bayesian Linear Regression - Implementation

In this problem, we will implement Bayesian Gaussian linear regression,
essentially reproducing the example [from lecture](https://davidrosenberg.github.io/mlcourse/Archive/2016/Lectures/13a.bayesian-regression.pdf\#page=12),
which in turn is based on the example in Figure 3.7 of Bishop's Pattern
Recognition and Machine Learning (page 155). We've provided plotting
functionality in "support_code.py". Your task is to complete "problem.py". The
implementation uses np.matrix objects, and you are welcome to use the np.matrix.getI method. 

**5.1** Implement likelihood\_func.

In [2]:
# Implement likelihood function

**5.2** Implement get\_posterior\_params.

In [3]:
# Implment get posterior params

**5.3** Implement get\_predictive\_params.

In [4]:
# Implement get predictive params

**5.4** Run ``python problem.py`` from inside the Bayesian Regression directory
to do the regression and generate the plots. This runs through the
regression with three different settings for the prior covariance.
You may want to change the default behavior in support\_code.make\_plots
from plt.show, to saving the plots for inclusion in your homework
submission.

In [5]:
# plots

**5.5** Comment on your results. In particular, discuss how each of the following
change with sample size and with the strength of the prior:  (i) the
likelihood function, (ii) the posterior distribution, and (iii) the
posterior predictive distribution.

**Answer**:

**5.6** Our work above was very much ``full Bayes``, in that rather than
coming up with a single prediction function, we have a whole distribution
over posterior prediction functions. However, sometimes we want a
single prediction function, and a common approach is to use the MAP
estimate -- that is, choose the prediction function that has the
highest posterior likelihood. As we discussed in class, for this setting,
we can get the MAP estimate using ridge regression. Use ridge regression
to get the MAP prediction function corresponding to the first prior
covariance ($\Sigma=\frac{1}{2}I$, per the support code). What value
did you use for the regularization coefficient? Why?

**Answer**: